# Task 14 – ETL Mini Pipeline

Python ETL using pandas and SQLite

In [ ]:

import pandas as pd
import sqlite3
import os


In [ ]:

# Create folders
os.makedirs("raw", exist_ok=True)
os.makedirs("processed", exist_ok=True)
os.makedirs("output", exist_ok=True)


In [ ]:

# Load dataset
df = pd.read_csv("raw/retail_sales.csv")
print("Raw Records:", df.shape)
df.head()


In [ ]:

# Data Cleaning
df = df.drop_duplicates()
df.fillna({"profit": 0, "discount": 0}, inplace=True)


In [ ]:

# Standardize columns
df.columns = df.columns.str.lower().str.replace(" ", "_")
df["order_date"] = pd.to_datetime(df["order_date"])


In [ ]:

# Derived columns
df["margin"] = (df["profit"] / df["sales"]).round(2)
df["is_high_value_customer"] = df["sales"].apply(lambda x: "Yes" if x > 1000 else "No")


In [ ]:

# Save processed data
df.to_csv("processed/processed_data.csv", index=False)


In [ ]:

# Split tables
customers = df[["customer_id", "customer_name", "segment", "region"]].drop_duplicates()
orders = df[["order_id", "order_date", "ship_mode", "customer_id", "sales", "profit", "margin"]]
products = df[["product_id", "product_name", "category", "sub_category"]].drop_duplicates()


In [ ]:

# Export CSVs
customers.to_csv("output/customers.csv", index=False)
orders.to_csv("output/orders.csv", index=False)
products.to_csv("output/products.csv", index=False)


In [ ]:

# Load to SQLite
conn = sqlite3.connect("database.sqlite")
customers.to_sql("customers", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)
conn.close()
